### Implementacion del motor de ingesta

Se ha compartido un archivo wheel el cual debe ser instalado en el computo del cluster, aquí procedemos a instalarlo:

Recordar que para esta parte se deben tener creados dos nuevos blob containers:

1. landingcaso
2. datalakecaso

Ya que en estos se realizará la ejecución de todo lo necesario.

#### Import para el proyecto

In [0]:
import sys
import os
import json
from confluent_kafka.schema_registry import SchemaRegistryClient

from MotorIngesta.batch_ingestion import batch_ingestion
from MotorIngesta.streaming_ingestion import streaming_ingestion

#### Archivo de configuración, lectura:

In [0]:
def read_config(file_path):
    with open(file_path, 'r') as file:
        config = json.load(file)
    return config

## Ingesta de datos batch

In [0]:
format = "json"

account = spark.conf.get("adls.account.name")

datasource = 'batch_operations'
dataset = "sales_orders"

landing_container = f"abfss://landingcaso@{account}.dfs.core.windows.net"
lakehouse_container = f"abfss://datalakecaso@{account}.dfs.core.windows.net"

landing_path = landing_container
raw_path = f"{lakehouse_container}/raw"
bronze_path = f"{lakehouse_container}/bronze"

dataset_landing_path = f"{landing_path}/{datasource}/{dataset}"
dataset_raw_path =  f"{raw_path}/{datasource}/{dataset}"
dataset_bronze_path = f"{bronze_path}/{datasource}/{dataset}"

print(landing_path)
print(raw_path)
print(bronze_path)

print(dataset_landing_path)
print(dataset_raw_path)
print(dataset_bronze_path)

In [0]:
batch_ingestion(datasource, dataset, landing_path, raw_path, bronze_path, format, spark=spark, dbutils=dbutils)

## Ingesta de datos desde Kafka

In [0]:
topic = "mobile_client"
datasource = "mobile"
dataset = topic

dataset_landing_path = f"{landing_path}/{datasource}/{dataset}"
dataset_raw_path =  f"{raw_path}/{datasource}/{dataset}"
dataset_bronze_path = f"{bronze_path}/{datasource}/{dataset}"
dataset_bronze_checkpoint_path = f"{bronze_path}/{datasource}/{dataset}_checkpoint"
table_name = f"hive_metastore.bronze.{datasource}_{dataset}"

config_dict = read_config('/dbfs/FileStore/config.json')
# print(config_dict)

schema_registry_url = config_dict["schema_registry_url"]
schema_registry_username = config_dict["schema_registry_username"]
schema_registry_password = config_dict["schema_registry_password"]

schema_registry_conf = {'url': schema_registry_url,
                        'basic.auth.user.info' : f'{schema_registry_username}:{schema_registry_password}'}
schema_registry_client = SchemaRegistryClient(schema_registry_conf)

print(dataset_landing_path)
print(dataset_raw_path)
print(dataset_bronze_path)
print(table_name)

In [0]:
streaming_ingestion(topic, dataset_bronze_path, dataset_bronze_checkpoint_path, table_name, spark, schema_registry_client)